<h3> Meesho HackerRank SQL Interview Question and Answer | Customer Budget </h3>

In [0]:
%sql
create table products
(
product_id varchar(20) ,
cost int
);
insert into products values ('P1',200),('P2',300),('P3',500),('P4',800);

create table customer_budget
(
customer_id int,
budget int
);

insert into customer_budget values (100,400),(200,800),(300,1500);

num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql
select * from products

product_id,cost
P1,200
P2,300
P3,500
P4,800


In [0]:
%sql
select * from customer_budget

customer_id,budget
100,400
200,800
300,1500


In [0]:
%sql
select customer_id,budget,count(*) as cnt_product,concat_ws(',', collect_list(product_id))as product from (
select *,sum(cost)over(partition by c.customer_id order by p.cost)running_sum from customer_budget c
cross join products p )a1
where budget>a1.running_sum
group by customer_id,budget

customer_id,budget,cnt_product,product
100,400,1,P1
200,800,2,"P1,P2"
300,1500,3,"P1,P2,P3"


<h3> solution in pyspark <\h3>

In [0]:
from pyspark.sql.functions import col,concat_ws,collect_list,count
from pyspark.sql.window import Window

In [0]:
df_p=spark.sql("select * from products")
df_c=spark.sql("select * from customer_budget")
df_p.show()
df_c.show()

+----------+----+
|product_id|cost|
+----------+----+
|        P1| 200|
|        P2| 300|
|        P3| 500|
|        P4| 800|
+----------+----+

+-----------+------+
|customer_id|budget|
+-----------+------+
|        100|   400|
|        200|   800|
|        300|  1500|
+-----------+------+



In [0]:
df_c.crossJoin(df_p).withColumn("running_sum",sum("cost").over(Window.partitionBy("customer_id").\
    orderBy("cost"))).filter(col("budget")>col("running_sum")).groupBy("customer_id","budget")\
        .agg(count("product_id").alias("cnt_product"),concat_ws(",", collect_list("product_id")).alias("product")).show()

+-----------+------+-----------+--------+
|customer_id|budget|cnt_product| product|
+-----------+------+-----------+--------+
|        100|   400|          1|      P1|
|        200|   800|          2|   P1,P2|
|        300|  1500|          3|P1,P2,P3|
+-----------+------+-----------+--------+

